In [ ]:
# Tout-en-un: installation, configuration, collecte, rapport (exécuter cette cellule unique)
!pip install -q pandas pyarrow requests openpyxl

from google.colab import drive
import requests
import pandas as pd
import datetime
import os
import logging
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import time
from getpass import getpass

# --- Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Mount Google Drive ---
try:
    drive.mount('/content/drive')
    logger.info("Google Drive mounted successfully.")
except Exception as e:
    logger.error(f"Error mounting Google Drive: {e}. Please check your connection and authentication.")

# --- API CoinMarketCap ---
try:
    API_KEY = getpass("Entre ta clé API CoinMarketCap (masquée): ")
    if not API_KEY:
        raise ValueError("Clé API vide.")
except Exception as e:
    raise RuntimeError(f"Erreur saisie de la clé API: {e}")

try:
    limit_input = input("Nombre de tokens à analyser (ex: 2000): ").strip()
    LIMIT = int(limit_input) if limit_input else 2000
    if LIMIT <= 0:
        raise ValueError
except Exception:
    LIMIT = 2000
    print("Valeur invalide, utilisation de LIMIT=2000 par défaut.")

API_URL = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?start=1&limit={LIMIT}&convert=USD'
HEADERS = {
    'X-CMC_PRO_API_KEY': API_KEY,
    'Accept': 'application/json'
}

# --- Chemins Google Drive ---
SAVE_DIR = "/content/drive/MyDrive/crypto_data"
REPORT_FILE = "/content/drive/MyDrive/rapport_performance_crypto.xlsx"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Fonctions ---

def get_weekly_parquet_filename():
    now = datetime.datetime.now()
    week_number = now.strftime("%G-W%V")
    return os.path.join(SAVE_DIR, f'crypto_data_wide_{week_number}.parquet')


def get_current_crypto_data():
    try:
        if API_KEY in (None, "", "REMPLACE_ICI_TA_CLE_API"):
            logger.error("API_KEY CoinMarketCap manquante: remplace API_KEY par ta clé réelle.")
            return None
        response = requests.get(API_URL, headers=HEADERS, timeout=30)
        response.raise_for_status()
        tokens = response.json().get('data', [])
        timestamp_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        data = [{
            'ID': t.get('id'),
            'Nom': t.get('name'),
            'Symbole': t.get('symbol'),
            'Prix (USD)': t['quote']['USD'].get('price'),
            'Timestamp': timestamp_str
        } for t in tokens]
        df = pd.DataFrame(data)
        df.sort_values(by='ID', inplace=True)
        logger.info(f"{len(df)} actifs reçus depuis CoinMarketCap.")
        return df
    except requests.exceptions.RequestException as e:
        logger.error(f"Erreur API : {e}")
        return None


def collect_data():
    CURRENT_PARQUET_FILE = get_weekly_parquet_filename()
    BASE_DF = None
    if os.path.exists(CURRENT_PARQUET_FILE):
        try:
            df_existing = pd.read_parquet(CURRENT_PARQUET_FILE)
            BASE_DF = df_existing[['ID', 'Nom', 'Symbole']].drop_duplicates(subset=['ID']).sort_values('ID').reset_index(drop=True)
        except Exception as e:
            logger.error(f"Erreur lecture {CURRENT_PARQUET_FILE}: {e}")

    df_new_data = get_current_crypto_data()
    if df_new_data is not None and not df_new_data.empty:
        if BASE_DF is None:
            BASE_DF = df_new_data[['ID', 'Nom', 'Symbole']].sort_values('ID').reset_index(drop=True)
            df_to_save = df_new_data.pivot_table(index=['ID', 'Nom', 'Symbole'], columns='Timestamp', values='Prix (USD)').reset_index()
            df_to_save.to_parquet(CURRENT_PARQUET_FILE)
            logger.info(f"Créé: {CURRENT_PARQUET_FILE}")
        else:
            df_new_base = df_new_data[['ID', 'Nom', 'Symbole']].sort_values('ID').reset_index(drop=True)
            df_merged_base = pd.merge(BASE_DF, df_new_base, on='ID', how='outer', suffixes=('', '_new'))
            df_merged_base['Nom'] = df_merged_base['Nom'].fillna(df_merged_base['Nom_new'])
            df_merged_base['Symbole'] = df_merged_base['Symbole'].fillna(df_merged_base['Symbole_new'])
            df_merged_base.drop(columns=['Nom_new', 'Symbole_new'], inplace=True, errors='ignore')
            df_merged_base.sort_values(by='ID', inplace=True)
            BASE_DF = df_merged_base.reset_index(drop=True)

            df_weekly_history = pd.read_parquet(CURRENT_PARQUET_FILE)
            df_to_append = df_new_data.pivot_table(index=['ID'], columns='Timestamp', values='Prix (USD)').reset_index()
            df_combined = pd.merge(df_weekly_history, df_to_append, on='ID', how='outer')
            final_df = pd.merge(BASE_DF, df_combined.drop(columns=['Nom', 'Symbole'], errors='ignore'), on='ID', how='outer')
            final_df.to_parquet(CURRENT_PARQUET_FILE)
            logger.info(f"Mise à jour: {CURRENT_PARQUET_FILE} (colonnes={len(final_df.columns)})")
    return CURRENT_PARQUET_FILE




def generate_performance_report(parquet_file: str, output_excel_file: str) -> bool:
    if not os.path.exists(parquet_file):
        logger.warning(f"Parquet introuvable: {parquet_file}")
        return False

    df = pd.read_parquet(parquet_file)

    price_columns = [c for c in df.columns if c not in ['ID', 'Nom', 'Symbole']]
    price_columns = [c for c in price_columns if not pd.isna(pd.to_datetime(c, errors='coerce'))]
    price_columns.sort(key=lambda x: pd.to_datetime(x))

    if len(price_columns) < 2:
        logger.info("Pas enough timestamps to calculate variations (>= 2 needed).")
        return False

    if df.empty:
        logger.info("No tokens to include in the report.")
        return False

    df_report = df[['ID', 'Nom', 'Symbole']].copy()
    first_price_col = price_columns[0]
    last_price_col = price_columns[-1]
    for i in range(1, len(price_columns)):
        current_col = price_columns[i]
        pct_change = ((df[current_col] - df[first_price_col]) / df[first_price_col]) * 100
        df_report[f"À {current_col}"] = pct_change

    # Filtre sur la dernière colonne et tri décroissant par % d'augmentation le plus récent
    last_col_name = f"À {last_price_col}"
    # S'assure que la colonne est bien numérique pour un tri correct
    df_report[last_col_name] = pd.to_numeric(df_report[last_col_name], errors='coerce')
    keep_rows = df_report[last_col_name] >= 1
    df_report_filtered = (
        df_report[keep_rows]
        .sort_values(by=last_col_name, ascending=False, na_position='last')
    )

    os.makedirs(os.path.dirname(output_excel_file), exist_ok=True)
    df_report_filtered.to_excel(output_excel_file, index=False)
    logger.info(f"Excel written: {output_excel_file} (rows={len(df_report_filtered)})")
    return True

# --- Main loop ---
while True:
    print("📥 Collecting data...")
    parquet_path = collect_data()
    print(f"Expected parquet: {parquet_path} -> exists={os.path.exists(parquet_path)}")

    print("📊 Generating report...")
    report_ok = generate_performance_report(parquet_path, REPORT_FILE)
    if report_ok:
        print(f"✅ Report updated: {REPORT_FILE}")
    else:
        print("ℹ️ Report not generated (conditions not met).")

    print("⏳ Pausing for 10 minutes...")
    time.sleep(600)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📥 Collecting data...


Expected parquet: /content/drive/MyDrive/crypto_data/crypto_data_wide_2025-W32.parquet -> exists=True
📊 Generating report...
✅ Report updated: /content/drive/MyDrive/rapport_performance_crypto.xlsx
⏳ Pausing for 10 minutes...
